In [7]:
%reload_ext autoreload
%autoreload 2

  Using cached pick-2.2.0-py3-none-any.whl (4.9 kB)
Note: you may need to restart the kernel to use updated packages.


In [28]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud

In [29]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

### Columns we can get from a slack message<br>

message_type, message_content, sender_id, time_sent, message_distribution, time_thread_start, reply_count, reply_user_count, time_thread_end, reply_users

From a single slack message, we can get <br>

1. The message<br>
2. Type (message, file, link, etc)<br>
3. The sender_id (assigned by slack)<br>
4. The time the message was sent<br>
5. The team (i don't know what that is now)<br>
6. The type of the message (broadcast message, inhouse, just messgae)<br>
7. The thread the message generated (from here we can go):<br>
    7.1 Text/content of the message<br>
    7.2 The thread time of the message<br>
    7.3 The thread count (reply count)<br>
    7.4 The number of user that reply the message (count of users that participated in the thread)<br>
    7.5 The time the last thread message was sent <br>
    7.6 The users that participated in the thread (their ids are stored as well)<br>

In [346]:
# combine all json file in all-weeks8-9

from src.loader import SlackDataLoader

# Initialize DataLoader
data_loader  = SlackDataLoader

# Load data from a Slack channel

slack_data = data_loader.slack_parser('../data/all-week5/')


Top 10 users by replaying

In [350]:
top_10_replaying_users = slack_data.groupby('sender_name')['reply_count'].sum().nlargest(10)
top_10_replaying_users

sender_name
Cristian Wilson     50
Brenda Hernandez    22
Mark Crawford       18
Phillip Atkins      11
Brian Odom          10
Brittany Garcia      8
Eric Myers           8
Travis Butler        8
Jodi Lyons           7
Kelly Soto           7
Name: reply_count, dtype: int64

Bottom 10  users by replying

In [225]:
bottom_10_replaying_users = slack_data.groupby('sender_name')['reply_count'].sum().nsmallest(10)

bottom_10_replaying_users


sender_name
Anthony Galloway    0
Brady Rhodes        0
David Scott         0
Jennifer Foley      0
John Romero         0
Joseph Diaz         0
Katherine Foster    0
Leah Newton         0
Luis Ingram         0
Mandy Sweeney       0
Name: reply_count, dtype: int64

Top 10  users by mention

In [230]:
top_10_mention_users = slack_data.groupby('sender_name')['msg_content'].apply(lambda x: x.str.count('@').sum()).nlargest(10)

top_10_mention_users


sender_name
Veronica Padilla    17
Brenda Hernandez    11
Cristian Wilson      7
Brian Odom           5
Katie Dickerson      5
Phillip Atkins       5
Carlos Gross         4
Joshua Rhodes        4
Katherine Foster     4
Michelle Lewis       4
Name: msg_content, dtype: int64

Bottom 10  users by mention

In [231]:
bottom_10_mention_users = slack_data.groupby('sender_name')['msg_content'].apply(lambda x: x.str.count('@').sum()).nsmallest(10)

bottom_10_mention_users


sender_name
Ashley Hamilton    0
Daniel Brown       0
David Scott        0
Dustin Wilson      0
Jodi Lyons         0
John Romero        0
Kelly Soto         0
Leah Newton        0
Luis Ingram        0
Melissa Ingram     0
Name: msg_content, dtype: int64

Top 10  users by message count

In [234]:
top_10_message_users = slack_data['sender_name'].value_counts().nlargest(10)

top_10_message_users

sender_name
Cristian Wilson     66
Brenda Hernandez    52
Travis Butler       45
Michelle Lewis      42
Veronica Padilla    35
Katie Dickerson     30
Joshua Rhodes       27
Brittany Garcia     24
Phillip Atkins      21
April Lucas         17
Name: count, dtype: int64

Bottom 10  users by message count

In [265]:
bottom_10_message_users = slack_data['sender_name'].value_counts().nsmallest(10)

bottom_10_mention_users

sender_name
Ashley Hamilton    0
Daniel Brown       0
David Scott        0
Dustin Wilson      0
Jodi Lyons         0
John Romero        0
Kelly Soto         0
Leah Newton        0
Luis Ingram        0
Melissa Ingram     0
Name: msg_content, dtype: int64

Top 10  users by reaction count

In [335]:
top_10_reaction_users = slack_data.groupby('sender_name')['reply_users_count'].sum().nlargest(10)

top_10_reaction_users

sender_name
Cristian Wilson     16
Brenda Hernandez    15
Brian Odom          10
Brittany Garcia      5
Eric Myers           5
Mark Crawford        5
Phillip Atkins       5
Jodi Lyons           4
Judith Bolton        4
Kelly Soto           4
Name: reply_users_count, dtype: int64

Bottom 10  users by reaction count

In [278]:
top_10_reaction_users = slack_data.groupby('sender_name')['reply_users_count'].sum().nlargest(10)

top_10_reaction_users

sender_name
Cristian Wilson     16
Brenda Hernandez    15
Brian Odom          10
Brittany Garcia      5
Eric Myers           5
Mark Crawford        5
Phillip Atkins       5
Jodi Lyons           4
Judith Bolton        4
Kelly Soto           4
Name: reply_users_count, dtype: int64

Top 10 messages by replies

In [354]:
top_10_messages_by_replies = slack_data.nlargest(10, 'reply_count')

top_10_messages_by_replies[['msg_content', 'reply_count']]


,msg_content,reply_count
20,"Hello, community .As I hear this advice from 1...",19
435,"hello guys, I have installed dbt successfully ...",12
138,"Has anyone managed to setup Airflow, Postgres ...",10
293,so can any one explain to me what is the next ...,10
226,can i use DBeaver as my DBMS? I was having iss...,9
133,---------------IMPORTANT----------------\nHi a...,8
343,How do we parse the .csv file? I don't underst...,8
1,Hello everyone...\n\nI really need help.. I'm ...,7
53,Every time I run airflow command my laptop rea...,7
128,is anyone here having trouble getting the data...,7


To 10 message by reaction

In [355]:
top_10_messages_by_reactions = slack_data.nlargest(10, 'reply_users_count')

top_10_messages_by_reactions[['msg_content', 'reply_users_count']]

,msg_content,reply_users_count
20,"Hello, community .As I hear this advice from 1...",5
133,---------------IMPORTANT----------------\nHi a...,5
128,is anyone here having trouble getting the data...,4
135,"<!here>, aws instance is being setup for every...",4
288,Do you know what session we have at 12:00 UTC?...,4
293,so can any one explain to me what is the next ...,4
40,I want to ask a question connected to Mohammed...,3
53,Every time I run airflow command my laptop rea...,3
146,I keep getting this error after installing doc...,3
226,can i use DBeaver as my DBMS? I was having iss...,3


Top 10 message by mention

In [345]:
slack_data['mention_count'] = slack_data['msg_content'].apply(lambda x: x.count('@'))

top_messages_by_mentions = slack_data.nlargest(10, 'mention_count')
top_messages_by_mentions[['msg_content', 'mention_count']]


,msg_content,mention_count
294,<@U03UVHCV6KB> <@U03UJGP0C68> <@U03UG4Q7V42> <...,4
286,<@U03U1FNPEUX> <@U03UH397319> <@U03UJGP0C68> T...,3
384,don't use any external downloader. Just use th...,3
74,<@U03UG32J3PC> <@U03U9EJR362> - How about now?...,2
83,<@U03UD68RQH3> and <@U03UUR571A5>\nI am not su...,2
102,<@U03U1FNPEUX> <@U03UJGP0C68> thank you both,2
226,can i use DBeaver as my DBMS? I was having iss...,2
269,I managed to solve my issue after manually ins...,2
274,<@U03U9EJR362> <@U03U1FNPEUX>?,2
395,<!here> join us in the Q/A session with <@U03U...,2


In [198]:
def get_top_20_user(data, channel='Random'):
    """get user with the highest number of message sent to any channel"""

    data['sender_name'].value_counts()[:20].plot.bar(figsize=(15, 7.5))
    plt.title(f'Top 20 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

    data['sender_name'].value_counts()[-10:].plot.bar(figsize=(15, 7.5))
    plt.title(f'Bottom 10 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

def draw_avg_reply_count(data, channel='Random'):
    """who commands many reply?"""

    data.groupby('sender_name')['reply_count'].mean().sort_values(ascending=False)[:20]\
        .plot(kind='bar', figsize=(15,7.5));
    plt.title(f'Average Number of reply count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_avg_reply_users_count(data, channel='Random'):
    """who commands many user reply?"""

    data.groupby('sender_name')['reply_users_count'].mean().sort_values(ascending=False)[:20].plot(kind='bar',
     figsize=(15,7.5));
    plt.title(f'Average Number of reply user count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_wordcloud(msg_content, week):    
    # word cloud visualization
    allWords = ' '.join([twts for twts in msg_content])
    wordCloud = WordCloud(background_color='#975429', width=500, height=300, random_state=21, max_words=500, mode='RGBA',
                            max_font_size=140, stopwords=stopwords.words('english')).generate(allWords)
    plt.figure(figsize=(15, 7.5))
    plt.imshow(wordCloud, interpolation="bilinear")
    plt.axis('off')
    plt.tight_layout()
    plt.title(f'WordCloud for {week}', size=30)
    plt.show()

def draw_user_reaction(data, channel='General'):
    data.groupby('sender_name')[['reply_count', 'reply_users_count']].sum()\
        .sort_values(by='reply_count',ascending=False)[:10].plot(kind='bar', figsize=(15, 7.5))
    plt.title(f'User with the most reaction in #{channel}', size=25);
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

## Insight Extraction

Below are some useful questions to answer. Feel free to explore to answer other interesting questions that may be of help to get insight about student's behaviour, need, and future performance 

In [ ]:
# which user has the highest number of reply counts?

In [ ]:
# Visualize reply counts per user per channel

In [ ]:
# what is the time range of the day that most messages are sent?


In [11]:
# what kind of messages are replied faster than others?

In [15]:
# Relationship between # of messages and # of reactions

In [16]:
# Classify messages into different categories such as questions, answers, comments, etc.

In [17]:
# Which users got the most reactions?

In [33]:
# Model topics mentioned in the channel

In [ ]:
# What are the topics that got the most reactions?

### Harder questions to look into

In [ ]:
# Based on messages, reactions, references shared, and other relevant data such as classification of questions into techical question, comment, answer, aorder stu the python, statistics, and sql skill level of a user?